In [1]:
from pytorch_h5dataset.benchmark import Benchmarker, BenchmarkDataset
from pytorch_h5dataset import H5DataLoader

from torch.utils.data import DataLoader
from torch import nn, float32, as_tensor
from torch.nn import MSELoss
from time import time
from numpy import prod
import seaborn as sns
from matplotlib import pyplot as plt
import psutil
import platform
cpu_count = 0 if platform.system() == 'Windows' else psutil.cpu_count()



benchmarkdataset = BenchmarkDataset(dataset_root="H:/Datasets/coco2017")

batch_size = 100
epochs = 100
device = 'cuda:0'

benchmarker1 = Benchmarker()
dataLoader1 = benchmarker1.decorate_iterator_class(H5DataLoader)(dataset=benchmarkdataset.h5dataset,
                                                                          device=device, batch_size=batch_size,
                                                                        return_meta_indices=True,
                                                                          pin_memory=True,
                                                                           num_workers=cpu_count)

benchmarker2 = Benchmarker()
dataloader2 = benchmarker2.decorate_iterator_class(DataLoader)(benchmarkdataset.imageFolderDataset,
                                                               batch_size=batch_size,
                                                               num_workers=cpu_count,
                                                               pin_memory=True)

criterion = MSELoss()


for benchmarker, dataloader in ((benchmarker1, dataLoader1) , (benchmarker2,dataloader2)):
    benchmarker.reset_benchmarker()
    model = nn.Linear(3 * 244 * 244, 1000).to(device)
    sum_loss = 0
    num_out = 0
    t0 = time()
    for e in range(epochs):
        print('\r',e, end='')
        for sample, label in dataloader:
            if isinstance(label, tuple):
                label = label[0]
            x = sample.to(device).view(sample.size(0),-1)
            y = as_tensor(label.view(-1), dtype=float32,device=device).requires_grad_(True)
            y_out = model(x).argmax(1).float()
            num_out += prod(y_out.shape)
            loss = criterion(y, y_out)
            loss = loss.sum()
            sum_loss += loss.item()
            loss.backward()

    print(f"Time for {epochs} epochs was {time() - t0}")
    print(x.min(),x.max())
    print(loss, num_out)
    #del dataloader, x,y, model, loss, sum_loss

 99Time for 100 epochs was 1079.5236911773682
tensor(-1., device='cuda:0') tensor(1., device='cuda:0')
tensor(295457.2812, device='cuda:0', grad_fn=<SumBackward0>) 500000
 1

KeyboardInterrupt: 

In [ ]:

df = benchmarker1.get_stats_df()
sns.set()
legend = ['proc_cpu_util','proc_disk_io_bytes_read','proc_disk_io_count_read','proc_cpu_time_user', 'proc_mem_bytes_vms', 'sys_net_io_bytes_recv']
plt.rcParams["figure.figsize"] = (15,5)
for col in legend:
    plt.plot(df[col]/ max(abs(df[col])))
plt.legend(legend)
plt.show()
legend = ['proc_cpu_util','proc_disk_io_bytes_read_acc','proc_disk_io_count_read_acc','proc_cpu_time_user_acc', 'proc_mem_bytes_vms_acc', 'sys_net_io_bytes_recv_acc']
plt.rcParams["figure.figsize"] = (15,5)
for col in legend:
    plt.plot(df[col])
plt.legend(legend)
plt.show()

In [ ]:
df = benchmarker2.get_stats_df()
sns.set()
legend = ['proc_cpu_util','proc_disk_io_bytes_read','proc_disk_io_count_read','proc_cpu_time_user', 'proc_mem_bytes_vms', 'sys_net_io_bytes_recv']
plt.rcParams["figure.figsize"] = (15,5)
for col in legend:
    plt.plot(df[col]/ max(abs(df[col])))
plt.legend(legend)
plt.show()
legend = ['proc_cpu_util','proc_disk_io_bytes_read_acc','proc_disk_io_count_read_acc','proc_cpu_time_user_acc', 'proc_mem_bytes_vms_acc', 'sys_net_io_bytes_recv_acc']
plt.rcParams["figure.figsize"] = (15,5)
for col in legend:
    plt.plot(df[col])
plt.legend(legend)
plt.show()


